# Simulate proteome time profiles.
## Start from measured proteome that is already hedged: necessary to enable lactate usage later?

# Using D Heckmann's kapps

## Select kapps associated with pathways relevant to the 6 C srcs since many others too low

In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
#pd.set_option('display.float_format', lambda x: '%.3g' % x)
%matplotlib inline

In [2]:
PROTOTYPE = False

In [3]:
from dynamicme.tests.helpme import HelpME
from dynamicme.dynamic import get_cplx_concs

helper = HelpME(prototyping=PROTOTYPE)
solver = helper.solver
me = solver.me

In [4]:
df_dh = pd.read_csv('/home/laurence/ME/data/DHeckmann/kappmax_rf_dl_iJO.csv')

In [5]:
import re
from cobra.io import load_json_model

ijo = load_json_model('/home/laurence/ME/models/BiGG_M/json/iJO1366.json')

rows = []
biggids = df_dh['bigg.id'].apply(lambda x: re.sub(r'(_[b|f]$)', '',x)).unique()
missed = []
for biggid in biggids:
    try:
        sdata = me.stoichiometric_data.get_by_id(biggid)
        mrxn = ijo.reactions.get_by_id(biggid)
        rname = mrxn.name
        for rxn in sdata.parent_reactions:
            rows.append({'bigg_id':biggid, 'rxn':rxn.id, 'name':rname, 'keff_nominal':rxn.keff})
    except KeyError:
        missed.append(biggid)
        
df_rmap = pd.DataFrame(rows)
        
print 'biggids not mapped:', len(missed)

biggids not mapped: 252


In [6]:
df_rmap.head(2)

,bigg_id,keff_nominal,name,rxn
0,12DGR120tipp,65.0,"1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C12:0)",12DGR120tipp_FWD_CPLX_dummy
1,12DGR140tipp,65.0,"1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C14:0)",12DGR140tipp_FWD_CPLX_dummy


In [7]:
df_manual = pd.merge(df_dh.rename(columns={'bigg.id':'bigg_id'}), df_rmap, on='bigg_id')

In [8]:
df_manual.head(2)

,bigg_id,kappmax.pred.rf.wo.impu,kappmax.pred.rf.w.impu,kappmax.pred.rf.w.impu.const.flux,kappmax.pred.rf.wo.impu.train.impu,kappmax.pred.rf.w.impu.train.impu,kappmax.pred.rf.w.impu.const.flux.train.impu,kappmax.pred.dl.wo.impu,kappmax.pred.dl.w.impu,kappmax.pred.dl.w.impu.const.flux,kappmax.pred.dl.wo.impu.train.impu,kappmax.pred.dl.w.impu.impu.train.impu,kappmax.pred.dl.w.impu.const.flux.impu.train.impu,experim_kappmax,keff_nominal,name,rxn
0,12DGR120tipp,8.046298,0.000039,0.000039,9.487162,0.000112,0.000112,1.066585,9.379017e-17,4.402115e-22,7.099318,0.130577,0.000007,NaN,65.0,"1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C12:0)",12DGR120tipp_FWD_CPLX_dummy
1,12DGR140tipp,8.046298,3.391126,0.000043,9.487162,3.277282,0.000114,1.066585,3.125016e+00,4.608598e-09,7.099318,66.749427,0.000001,NaN,65.0,"1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C14:0)",12DGR140tipp_FWD_CPLX_dummy


In [9]:
COL_KEFF = 'kappmax.pred.rf.wo.impu'
df_manual.loc[:,'keff'] = df_manual[COL_KEFF]

In [10]:
df_tuned = df_manual[ df_manual.keff != df_manual.keff_nominal]
print df_tuned.shape
df_tuned

(1857, 18)


,bigg_id,kappmax.pred.rf.wo.impu,kappmax.pred.rf.w.impu,kappmax.pred.rf.w.impu.const.flux,kappmax.pred.rf.wo.impu.train.impu,kappmax.pred.rf.w.impu.train.impu,kappmax.pred.rf.w.impu.const.flux.train.impu,kappmax.pred.dl.wo.impu,kappmax.pred.dl.w.impu,kappmax.pred.dl.w.impu.const.flux,kappmax.pred.dl.wo.impu.train.impu,kappmax.pred.dl.w.impu.impu.train.impu,kappmax.pred.dl.w.impu.const.flux.impu.train.impu,experim_kappmax,keff_nominal,name,rxn,keff
0,12DGR120tipp,8.046298,0.000039,0.000039,9.487162,0.000112,0.000112,1.066585,9.379017e-17,4.402115e-22,7.099318,0.130577,6.932758e-06,NaN,65.000000,"1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C12:0)",12DGR120tipp_FWD_CPLX_dummy,8.046298
1,12DGR140tipp,8.046298,3.391126,0.000043,9.487162,3.277282,0.000114,1.066585,3.125016e+00,4.608598e-09,7.099318,66.749427,1.415470e-06,NaN,65.000000,"1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C14:0)",12DGR140tipp_FWD_CPLX_dummy,8.046298
2,12DGR141tipp,8.046298,11.947018,0.000145,9.487162,32.300362,0.000660,1.066585,2.259487e+01,7.142521e-08,7.099318,34.810117,3.375369e-07,NaN,65.000000,"1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C14:1)",12DGR141tipp_FWD_CPLX_dummy,8.046298
3,12DGR160tipp,8.046298,8.774145,0.001198,9.487162,11.518131,0.001138,1.066585,1.429035e+01,3.110135e-07,7.099318,9.989350,7.255580e-07,NaN,65.000000,"1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C16:0)",12DGR160tipp_FWD_CPLX_dummy,8.046298
4,12DGR161tipp,8.046298,0.246542,0.001043,9.487162,0.295304,0.000985,1.066585,1.815298e+00,1.435127e-07,7.099318,2.897581,3.479175e-07,NaN,65.000000,"1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C16:1)",12DGR161tipp_FWD_CPLX_dummy,8.046298
5,12DGR180tipp,8.046298,9.426349,0.000066,9.487162,11.533059,0.000302,1.066585,7.946862e+00,4.138248e-08,7.099318,0.248352,1.119554e-07,NaN,65.000000,"1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C18:0)",12DGR180tipp_FWD_CPLX_dummy,8.046298
6,12DGR181tipp,8.046298,4.346991,0.000938,9.487162,6.954457,0.001158,1.066585,4.458432e+00,3.833023e-08,7.099318,106.414822,1.149125e-06,NaN,65.000000,"1,2 diacylglycerol transport via flipping (periplasm to cytoplasm, n-C18:1)",12DGR181tipp_FWD_CPLX_dummy,8.046298
7,14GLUCANabcpp,8.046298,2.330346,2.330346,9.487162,8.349860,8.349860,0.312369,1.093099e-01,1.093099e-01,1.954541,9.495512,9.495512e+00,NaN,117.900768,"1,4-alpha-D-glucan transport via ABC system (periplasm)",14GLUCANabcpp_FWD_ABC-16-CPLX,8.046298
8,14GLUCANtexi,8.046298,6.924344,6.924344,9.487162,8.861699,8.861699,0.650559,7.589476e-01,7.589476e-01,6.357923,5.732537,5.732537e+00,NaN,89.990608,"1,4-alpha-D-glucan transport via diffusion (extracellular to periplasm) irreversible",14GLUCANtexi_FWD_CPLX0-7655,8.046298
9,23DAPPAt2pp,8.046298,6.899793,6.899793,9.487162,11.422289,11.422289,8.399925,6.598027e+00,6.598027e+00,33.263320,39.949041,3.994904e+01,NaN,65.000000,"2,3-diaminopropionate transport in via proton symport",23DAPPAt2pp_FWD_CPLX_dummy,8.046298


In [11]:
from cobra.io import load_json_model
import numpy as np

ijo = load_json_model('/home/laurence/ME/models/BiGG_M/json/iJO1366.json')

for rxn in me.reactions.query('LDH'):
    keff = rxn.keff if hasattr(rxn,'keff') else np.nan
    sid = rxn.stoichiometric_data.id if hasattr(rxn,'stoichiometric_data') else ''    
    sname = ''
    if sid != '':
        sname = ijo.reactions.get_by_id(sid).name
    print rxn.id, '\t', rxn.x, '\t', keff, '\t', sid, '\t', sname

formation_ALDHDEHYDROG-MONOMER 	0.0 	nan 	 	
ALDD2x_FWD_ALDHDEHYDROG-MONOMER 	0.0 	41.5459418172 	ALDD2x 	Aldehyde dehydrogenase (acetaldehyde, NAD)
APPLDHr_REV_GLYCDEH-CPLX 	0.0 	155.171732401 	APPLDHr 	Aminopropanol dehydrogenase reversible
APPLDHr_FWD_GLYCDEH-CPLX 	0.0 	155.171732401 	APPLDHr 	Aminopropanol dehydrogenase reversible
BETALDHx_FWD_BADH-CPLX 	0.0 	116.667653266 	BETALDHx 	Betaine-aldehyde dehydrogenase
BETALDHy_FWD_BADH-CPLX 	0.0 	116.667653266 	BETALDHy 	Betaine-aldehyde dehydrogenase
FALDH2_REV_ADHC-CPLX_mod_zn2 	0.0 	55.5610475968 	FALDH2 	Formaldehyde dehydrogenase
FALDH2_FWD_ADHC-CPLX_mod_zn2 	0.0 	55.5610475968 	FALDH2 	Formaldehyde dehydrogenase
GGGABADr_REV_ALDHDEHYDROG-MONOMER 	0.0 	41.5459418172 	GGGABADr 	Gamma-glutamyl-gamma aminobutyric acid dehydrogenase
GGGABADr_FWD_ALDHDEHYDROG-MONOMER 	0.0 	41.5459418172 	GGGABADr 	Gamma-glutamyl-gamma aminobutyric acid dehydrogenase
LDH_D_REV_DLACTDEHYDROGFAD-MONOMER_mod_fad 	0.0 	47.9209545292 	LDH_D 	D-lactate dehydr

In [12]:
print '%-25.20s%-25.20s%-25s' % ('keff0','keff (tuned)', 'rxn')
for i,r in df_tuned.iterrows():
    rxn = me.reactions.get_by_id(r['rxn'])
    print '%-25.20s%-25.20s%-25s' % (rxn.keff, r['keff'], rxn.id)
    print 'Setting to tuned keff'
    rxn.keff = r['keff']
    rxn.update()

keff0                    keff (tuned)             rxn                      
65                       8.04629791189            12DGR120tipp_FWD_CPLX_dummy
Setting to tuned keff
65                       8.04629791189            12DGR140tipp_FWD_CPLX_dummy
Setting to tuned keff
65                       8.04629791189            12DGR141tipp_FWD_CPLX_dummy
Setting to tuned keff
65                       8.04629791189            12DGR160tipp_FWD_CPLX_dummy
Setting to tuned keff
65                       8.04629791189            12DGR161tipp_FWD_CPLX_dummy
Setting to tuned keff
65                       8.04629791189            12DGR180tipp_FWD_CPLX_dummy
Setting to tuned keff
65                       8.04629791189            12DGR181tipp_FWD_CPLX_dummy
Setting to tuned keff
117.900767577            8.04629791189            14GLUCANabcpp_FWD_ABC-16-CPLX
Setting to tuned keff
89.9906082897            8.04629791189            14GLUCANtexi_FWD_CPLX0-7655
Setting to tuned keff
65                    

In [13]:
print '%-25.20s%-25.20s%-25s' % ('keff0','keff (tuned)', 'rxn')
for i,r in df_tuned.iterrows():
    rxn = me.reactions.get_by_id(r['rxn'])
    print '%-25.20s%-25.20s%-25s' % (rxn.keff, r['keff'], rxn.id)    
    print rxn.reaction

keff0                    keff (tuned)             rxn                      
8.04629791189            8.04629791189            12DGR120tipp_FWD_CPLX_dummy
12dgr120_p + 3.45224326540611e-5*mu CPLX_dummy --> 12dgr120_c
8.04629791189            8.04629791189            12DGR140tipp_FWD_CPLX_dummy
12dgr140_p + 3.45224326540611e-5*mu CPLX_dummy --> 12dgr140_c
8.04629791189            8.04629791189            12DGR141tipp_FWD_CPLX_dummy
12dgr141_p + 3.45224326540611e-5*mu CPLX_dummy --> 12dgr141_c
8.04629791189            8.04629791189            12DGR160tipp_FWD_CPLX_dummy
12dgr160_p + 3.45224326540611e-5*mu CPLX_dummy --> 12dgr160_c
8.04629791189            8.04629791189            12DGR161tipp_FWD_CPLX_dummy
12dgr161_p + 3.45224326540611e-5*mu CPLX_dummy --> 12dgr161_c
8.04629791189            8.04629791189            12DGR180tipp_FWD_CPLX_dummy
12dgr180_p + 3.45224326540611e-5*mu CPLX_dummy --> 12dgr180_c
8.04629791189            8.04629791189            12DGR181tipp_FWD_CPLX_dummy
12dgr1

In [14]:
cplx_concs_max = get_cplx_concs(solver)

In [15]:
mu_me = me.solution.x_dict['biomass_dilution']
print 'mu of ME for growth maximization:', mu_me

mu of ME for growth maximization: 0.911689453125


In [16]:
from dynamicme.dynamic import DelayedDynamicME

dyme = DelayedDynamicME(solver)
dyme.delay_model.solver.set_realopts('lp', {'Feasibility tol':1e-15,'Optimality tol':1e-15})
dsolver = dyme.delay_model.solver

/home/laurence/python_modules/dynamicme.git/dynamicme/dynamic.py:1229 UserWarning: Dynamic mRNA not yet implemented!


## Initial condition proteome
1. Optimal predicted proteome on glucose (unrealistic--useful for illustrating difference from actual generalist observed)
1. Measured proteome on glucose (more realistic--generalist)

**A result in itself?:** explaining and predicting differences in growth trajectories depending on initial proteome
- explains why passing a few generations allows adaptation without mutation enabling faster growth of even the wild-type
- also means initial proteome definitely not at growth optimal--can show the difference
- does this mechanism also explain lag phase?

### Need to show results for multiple initial representative proteomes
1. Glucose optimal
1. Glycerol optimal (since glycerol wasn't fully utilized)
1. Measured glucose proteome--at fixed measured growth rate and closest proteome

# Initial proteome A: closest to measured proteome at measured growth rate
## Test the optimal predicted proteome

In [17]:
C_meas = 'Glucose'
###
df_meta = pd.read_csv('/home/laurence/ME/data/cobrame/sectors/proteomics_conds.csv')
mu_meas = df_meta[ df_meta.exp==C_meas].iloc[0]['Growth.rate..h.1.']
print 'Growth on %s = %g' % (C_meas, mu_meas)

Growth on Glucose = 0.58


In [18]:
df_meas = pd.read_csv('/home/laurence/ME/data/cobrame/sectors/schmidt_prot_mf.csv')
print df_meas.shape
#df_meas.head(2)

(51898, 8)


In [19]:
df_meas_C = df_meas[ df_meas.condition==C_meas]
#df_meas_C.head(2)

In [20]:
meas_dict = {r['Bnumber']:r['mf'] for i,r in df_meas_C.iterrows()}

In [21]:
sum(meas_dict.values())

0.9999999999999964

# Get simulated proteome closest to measured
- L1 (drops potential outliers)
- L2 (tries to get match everything)
    - should pre-process to remove outliers?

In [22]:
%%time

from cobrawe.fit import FitME

fit = FitME(me)
#me_nlp = fit.me_nlp
#me_nlp.compiled_expressions = None
#sol,stat,hs = fit.fit_mass_frac(meas_dict, mu_meas, norm='l1')

CPU times: user 73.3 ms, sys: 20 ms, total: 93.3 ms
Wall time: 91.2 ms


## Initialize dynamicME with this feasible initial proteome and growth rate

In [23]:
cplx_concs = get_cplx_concs(solver)
cplx_conc_dict0 = dict(cplx_concs)

# Validate mixed substrate uptake hierarchy--Beg et al. (2007) MSB

In [24]:
import warnings

T = 10   # hours
V = 1.   # L
X0 = 0.00675 / V  # g/L
c0_dict = {'glc__D_e': 0.4,
           'lac__L_e': 0.4,
           'malt_e': 0.4,
           'gal_e': 0.4,
           'glyc_e':0.4,
           'ac_e': 0.0}

### Convert from g/L to mmol
mids_remove = []
for mid,c in c0_dict.iteritems():
    try:        
        met = me.metabolites.get_by_id(mid)    
        c0_dict[met.id] = c / met.formula_weight * 1000
    except KeyError:
        warnings.warn('Model has no metabolite: %s. Will remove it from c0_dict' % mid)
        mids_remove.append(mid)

for mid in mids_remove:
    c0_dict.pop(mid)
        
### Set max uptake rate
LB_EX = -10.
LB_O2 = -20.

lb_dict={}
ub_dict={}

for mid in c0_dict.keys():
    try:
        rxn = dyme.get_exchange_rxn(mid)    
        if rxn.id is 'EX_o2_e':
            lb = LB_O2
        else:
            lb = LB_EX
        rxn.lower_bound = lb
        lb_dict[rxn.id] = lb
    except KeyError:
        warnings.warn('Model has no metabolite: '+ mid)

me.reactions.EX_o2_e.lower_bound = -20.
lb_dict[me.reactions.EX_o2_e.id] = -20.
        
print c0_dict
print lb_dict

{'glyc_e': 4.34339676647141, 'malt_e': 1.1685776026677226, 'lac__L_e': 4.4908498933423155, 'gal_e': 2.220299442904667, 'ac_e': 0.0, 'glc__D_e': 2.220299442904667}
{u'EX_glyc_e': -10.0, u'EX_o2_e': -20.0, u'EX_gal_e': -10.0, u'EX_lac__L_e': -10.0, u'EX_malt_e': -10.0, u'EX_ac_e': -10.0, 'EX_glc__D_e': -10.0}


In [25]:
import numpy as np
cvals = np.array(cplx_conc_dict0.values())
cvals[ cvals>0].min()

1.1849868116696687e-13

In [26]:
solver.set_realopts('lp', {'Feasibility tol':1e-15, 'Optimality tol':1e-15})

In [27]:
from cobrame import MetabolicReaction

if PROTOTYPE:
    fluxes_plot = ['ACONTa_CPLX0-7760_mod_4fe4s', 'ACONTb_CPLX0-7760_mod_4fe4s', 'ACKr_reverse']
else:
    #fluxes_plot = [r.id for r in dsolver.me.reactions if isinstance(r,MetabolicReaction)]
    fluxes_plot = [r.id for r in dsolver.me.reactions]

In [39]:
len(fluxes_plot)

15535

In [ ]:
%%time
extra_rxns_tracked = list(set(['EX_o2_e'] + fluxes_plot + ['biomass_dilution']))

T = 30
dt = 2
H = dt

result = dyme.simulate_batch(
    T,    
    c0_dict,
    X0,
    prec_bs=1e-3,
    ZERO_CONC=0.,
    extra_rxns_tracked=extra_rxns_tracked,
    lb_dict={},
    verbosity=2,
    dt = dt,
    H = H,
    cplx_conc_dict0=cplx_conc_dict0)

Metabolite glyc_e available.
Using default LB=-1000 for EX_glyc_e
Metabolite ac_e depleted.
Metabolite lac__L_e available.
Using default LB=-1000 for EX_lac__L_e
Metabolite malt_e available.
Using default LB=-1000 for EX_malt_e
Metabolite gal_e available.
Using default LB=-1000 for EX_gal_e
Metabolite glc__D_e available.
Using default LB=-1000 for EX_glc__D_e
Computing new uptake rates
Biomass at t=2: 0.0155039
Concentrations: {'glyc_e': 4.3434474860753802, 'ac_e': 0.0, 'lac__L_e': 3.9188455599654484, 'malt_e': 1.1685776026677226, 'gal_e': 2.220299442904667, 'glc__D_e': 1.8780417816102204}
Growth rate: 0.6484375
Metabolite glyc_e available.
Using default LB=-1000 for EX_glyc_e
Metabolite ac_e depleted.
Metabolite lac__L_e available.
Using default LB=-1000 for EX_lac__L_e
Metabolite malt_e available.
Using default LB=-1000 for EX_malt_e
Metabolite gal_e available.
Using default LB=-1000 for EX_gal_e
Metabolite glc__D_e available.
Using default LB=-1000 for EX_glc__D_e
Computing new upta

### Save results

In [ ]:
import json
result2 = result.copy()
result2.keys()
result2.pop('basis')

with open('/home/laurence/ME/data/dynamicME/production/validation1_uptake_hier_sim_opt0_longH_R03_DHkapp.json','w') as f:
    json.dump(result2, f)

In [ ]:
import seaborn as sns
%matplotlib inline

plt.plot(result['time'], result['biomass'])

In [ ]:
ex_met_rxn_dict = {m: {'rxn':dyme.get_exchange_rxn(m), 'mw':me.metabolites.get_by_id(m).formula_weight} for m in c0_dict.keys()}
ex_met_rxn_dict

In [ ]:
import pandas as pd

df_conc = pd.DataFrame(result['concentration'])
### Convert concentrations to g/L
for mid, vals in ex_met_rxn_dict.iteritems():
    df_conc[mid] = df_conc[mid] * vals['mw'] * 0.001

df_time = pd.DataFrame({'time':t, 'biomass':b} for t,b in zip(result['time'], result['biomass']))
df_flux = pd.DataFrame(result['ex_flux'])
df_mu   = pd.DataFrame({'mu':result['growth_rate']})
df_result = pd.concat([df_time, df_conc, df_flux, df_mu], axis=1)

In [ ]:
dsplot = pd.melt(df_result, id_vars='time', value_vars=['biomass','ac_e','gal_e','glc__D_e','glyc_e','lac__L_e','malt_e'])
dsplot.time = pd.to_numeric(dsplot.time)
dsplot['category'] = 'Metabolite'
dsplot.loc[ dsplot.variable=='biomass', 'category'] = 'Biomass'

In [ ]:
colors = [
'#000000', # biomass
'#660066', # acetate
'#00CC00', #'#00FF00', # galactose
'#000000', # glucose
'#FF00FF', # glycerol
'#0000FF', # lactate
'#FF0000', # maltose
]

colpal = sns.color_palette(colors)
sns.set_palette(colpal)

g = sns.FacetGrid(data=dsplot, hue='variable', col='category', sharey=False, size=5)

g.map(plt.plot, 'time', 'value', linestyle='-', marker='.')
for i,ax in enumerate(g.axes.flat):
    ylim = ax.get_ylim()
    ax.set_ylim((ylim[0],ylim[1]*1.1))
    if i==0:
        ax.set_ylabel('Biomass (g/L)')
    else:
        ax.set_ylabel('Metabolite concentration (g/L)')
g.set_xlabels('Time (h)')
g.add_legend()

In [ ]:
g.savefig('/home/laurence/ME/figures/persistME/beg_init_opt_longH_R03_DHkapp.pdf')
g.savefig('/home/laurence/ME/figures/persistME/beg_init_opt_longH_R03_DHkapp.svg')
g.savefig('/home/laurence/ME/figures/persistME/beg_init_opt_longH_R03_DHkapp.eps')

# Test/Debug
Take the last proteome and try to grow on left over substrates

In [ ]:
conc_dict_fin = result['complex'][-1]

dyme.delay_model.update_cplx_concs(conc_dict_fin)

In [ ]:
for rxn in dsolver.me.reactions.query('EX_'):
    if rxn.lower_bound < 0:
        x = result['rxn_flux'][-1][rxn.id]
        print '%-15.15s%-25.20s%-35s' % (x, rxn.lower_bound, rxn.id)

muopt, hs, xopt, cache = dsolver.bisectmu(basis=dsolver.feas_basis)

In [ ]:
if PROTOTYPE:
    complexes_plot = ['IPPMIa_E_c', 'ACKr_reverse_E_c', 'CPLX0-7760_mod_4fe4s','CPLX0-7761_mod_4fe4s']
else:
    complexes_plot = ['3-ISOPROPYLMALISOM-CPLX_mod_4fe4s', 'ACETATEKINA-MONOMER_mod_mg2', 'CPLX0-7659']

In [ ]:
len(fluxes_plot)

In [ ]:
fluxes_plot2 = []

for rid in fluxes_plot:
    rxn = dsolver.me.reactions.get_by_id(rid)
    x   = result['rxn_flux'][0][rid]
    try:
        if abs(x) > 1:
            fluxes_plot2.append(rxn.id)
    except:
        pass
        
print len(fluxes_plot2)

In [ ]:
df_mu = pd.DataFrame([{'time':t, 'variable':'growth_rate', 'value':c, 'category':'growth'} for t,c in zip(result['time'], result['growth_rate'])] )

df_complex = pd.DataFrame([{'time':t, 'complex':c, 'abundance':cdict[c]} 
                           for t,cdict in zip(result['time'], result['complex']) for c in complexes_plot] )

df_mflux = pd.DataFrame([{'time':t, 'rxn':c, 'flux':cdict[c]} 
                           for t,cdict in zip(result['time'], result['rxn_flux']) for c in fluxes_plot2] )

df_complex.loc[:,'category'] = 'complex'
try:
    df_mflux.loc[:,'category'] = 'flux'
except:
    pass

dsplot_all = pd.concat([dsplot,
                        df_mu,
                          df_complex.rename(columns={'complex':'variable', 'abundance':'value'}),
                          df_mflux.rename(columns={'rxn':'variable', 'flux':'value'})])

In [ ]:
sns.set_style('ticks')
palette = sns.color_palette('muted', len(dsplot_all.variable.unique()))
g = sns.FacetGrid(data=dsplot_all, hue='variable', sharey=False, size=3, col='category', palette=palette,
                  aspect=2, col_wrap=2, col_order=['Biomass','complex','Metabolite','flux','growth'])
g.map(plt.plot, 'time', 'value', linestyle='-', marker='.')
g.add_legend()

### In general, it seems virtual cell lacks capacity to replenish the continuously diluting proteome.
### Actually, not entirely true--see very rapid rise in ACK protein. But is that because it's cheaper (recall Hwa's overflow mechanism--ATP per protein cost)
### Need to start it off with much more ribosome?

## Proteome mol (mass) fractions

In [ ]:
#df_super = pd.read_csv('/home/laurence/ME/data/cobrame/proteome_letter/subsystem-supersystem-mapping.csv')
df_system = pd.read_csv('/home/laurence/ME/data/cobrame/proteome_letter/sub_supersystem.csv')
df_system.shape

# df_trsl = pd.DataFrame([{'rxn':k, 'v':v, 'time':result['time'][i]} for i,r in enumerate(result['rxn_flux']) for k,v in r.iteritems() if 'translation_' in k])

import re
patt = re.compile(r"protein_(b\d\d\d\d)_?\S?")

cplx_prot = pd.DataFrame([{'cplx':d.id, 'locus':re.findall(patt,k)[0]} for d in me.complex_data for k,v in d.stoichiometry.iteritems() if 'protein_b' in k])

df_mass = pd.DataFrame([{'cplx':met.id, 'mass':met.mass} for met in me.metabolites if hasattr(met,'mass')])
df_cplx = pd.DataFrame([{'cplx':k, 'conc':v, 'time':result['time'][i]} for i,r in enumerate(result['complex']) for k,v in r.iteritems()])
df_cplx_mass = pd.merge(df_cplx, df_mass, on='cplx')

times = df_cplx_mass.time.unique()
for t in times:
    dfi = df_cplx_mass[ df_cplx_mass.time==t]
    masses = dfi.conc*dfi.mass
    mtot = masses.sum()
    mfs = masses/mtot
    b = (df_cplx_mass.time==t).values
    df_cplx_mass.loc[b,'mf'] = mfs

df_cplx_mass = pd.merge(df_cplx_mass, cplx_prot, on='cplx', how='left')

In [ ]:
#syslevel = 'Subsystem'
syslevel = 'Supersystem'

df_level = df_system[['gene',syslevel]].drop_duplicates()

df_cplx_sub = pd.merge(df_cplx_mass, df_level, left_on='locus', right_on='gene', how='left')
df_cplx_sub.loc[ df_cplx_sub[syslevel].isnull(), syslevel] = 'Ungrouped'

df_mf_sector = df_cplx_sub.groupby(['time',syslevel]).mf.sum().reset_index()
# Renormalize to 1
dsplot_sector = df_mf_sector.copy()
for t in dsplot_sector.time.unique():
    b = dsplot_sector.time==t
    mtot = sum(dsplot_sector.loc[b,'mf'])
    dsplot_sector.loc[b, 'mf'] = dsplot_sector.loc[b, 'mf']/mtot

from stressme.plot import stacked_bar

dsplot_sector.loc[:,'group'] = syslevel
g = stacked_bar('time', 'mf', dsplot_sector, syslevel, 'group', size=5, aspect=3, palette_name='Paired')

In [ ]:
df_mass = pd.DataFrame([{'cplx':met.id, 'mass':met.mass} for met in me.metabolites if hasattr(met,'mass')])
df_cplx = pd.DataFrame([{'cplx':k, 'conc':v, 'time':result['time'][i]} for i,r in enumerate(result['complex']) for k,v in r.iteritems()])
df_cplx_mass = pd.merge(df_cplx, df_mass, on='cplx')

times = df_cplx_mass.time.unique()
for t in times:
    dfi = df_cplx_mass[ df_cplx_mass.time==t]
    masses = dfi.conc*dfi.mass
    mtot = masses.sum()
    mfs = masses/mtot
    b = (df_cplx_mass.time==t).values
    df_cplx_mass.loc[b,'mf'] = mfs

#df_var = df_cplx_mass.groupby('cplx').mf.var().reset_index()
df_var = df_cplx_mass.groupby('cplx').mf.mean().reset_index()
N_PLOT = 10
vary_cplxs = df_var.sort_values('mf', ascending=False)['cplx'][0:N_PLOT].values
vary_cplxs

df_cplx_mass[ df_cplx_mass.time==0].mf.sum()

from stressme.plot import stacked_bar

dsplot_cplx = df_cplx_mass[ df_cplx_mass.cplx.isin(vary_cplxs)]
dsplot_cplx.loc[:,'group'] = 'DynamicDelayedME'
stacked_bar('time', 'mf', dsplot_cplx, 'cplx', 'group', aspect=4, palette_name='Paired')